In [13]:
import requests
from pathlib import Path

SOURCE_PAGE = "https://www.insee.fr/fr/information/2120875"
PDF_URL = "https://www.insee.fr/fr/statistiques/fichier/2120875/Nomenclatures_NAF_et_CPF_Reedition_2020.pdf"

BASE_DIR = Path("./data/raw/temps/nafs")
PDF_PATH = BASE_DIR / "Nomenclatures_NAF_et_CPF_Reedition_2020.pdf"

BASE_DIR.mkdir(parents=True, exist_ok=True)

print("📥 Téléchargement du PDF NAF / CPF (INSEE): 'Nomenclature d’activités française – NAF rév. 2'")
print(f"🔗 Source officielle : {SOURCE_PAGE}")

r = requests.get(PDF_URL)
r.raise_for_status()

with open(PDF_PATH, "wb") as f:
    f.write(r.content)

print(f"✅ PDF téléchargé : {PDF_PATH}")


📥 Téléchargement du PDF NAF / CPF (INSEE): 'Nomenclature d’activités française – NAF rév. 2'
🔗 Source officielle : https://www.insee.fr/fr/information/2120875
✅ PDF téléchargé : data\raw\temps\nafs\Nomenclatures_NAF_et_CPF_Reedition_2020.pdf


In [14]:
# Ce programme extrait les descriptions des codes NAFS depuis le fichier PDF de l'INSEE

import pdfplumber
import re

def read_pdf_text(pdf_path):
    """
    Lit tout le PDF et retourne le texte complet.
    """
    with pdfplumber.open(pdf_path) as pdf:
        full_text = "\n".join(page.extract_text() or "" for page in pdf.pages)
    return full_text


def extract_best_naf_block(full_text, naf_code):
    """
    Renvoie le bloc NAF contenant la description complète,
    même s'il existe plusieurs occurrences (sommaire + section complète).
    """

    naf_code_clean = naf_code.upper().strip().replace("Z", "")  # ex: "05.10"

    title_regex = re.compile(r"^\s*\d{2}\.\d{2}[A-Z]?", re.MULTILINE)

    titles = [(m.group().strip(), m.start()) for m in title_regex.finditer(full_text)]

    # Trouver toutes les occurrences possibles
    candidate_indices = []
    for i, (title, _) in enumerate(titles):
        if title.replace("Z", "") == naf_code_clean:
            candidate_indices.append(i)

    if not candidate_indices:
        return None

    blocks = []

    # Extraire tous les blocs possibles
    for idx in candidate_indices:
        start = titles[idx][1]
        end = titles[idx+1][1] if idx+1 < len(titles) else len(full_text)
        block = full_text[start:end].strip()
        blocks.append(block)

    # Retourner le bloc le plus long → la vraie définition complète
    return max(blocks, key=len)

naf = "05.10Z"
full_text_pdf_naf = read_pdf_text(PDF_PATH)

In [15]:
# Exemple de sortie pour le code 05.10Z
extract_best_naf_block(full_text_pdf_naf, "05.10Z")

"05.10Z Extraction de houille\nCette sous-classe comprend :\n- l'extraction de la houille en souterrain ou à ciel ouvert, y compris par le recours à des procédés\nde liquéfaction\n- le lavage, le triage, le calibrage, la pulvérisation, la compression, etc. destinées à classer, à\naccroître la qualité ou à faciliter le transport et le stockage de la houille pour compte propre\nCette sous-classe comprend aussi :\n- l'exploitation des terrils en vue de la récupération de la houille\nCette sous-classe ne comprend pas :\n- l'extraction de lignite (cf. 05.20Z)\n- l'extraction de la tourbe (cf. 08.92Z)\n- les activités de soutien à l'extraction de la houille (cf. 09.90Z)\n- les forages d'essai pour l'extraction de la houille (cf. 09.90Z)\n- les fours à coke produisant des combustibles solides (cf. 19.10Z)\n- l'agglomération de la houille et la fabrication de briquettes de houille (cf. 19.20Z)\n46 Nomenclature d’Activités Française NAF rév. 2\nSection B : PRODUITS DES INDUSTRIES EXTRACTIVES\n0

In [16]:
# Exemple de sortie pour le code 85.51Z
extract_best_naf_block(full_text_pdf_naf, "85.51Z")

"85.51Z Enseignement de disciplines sportives et d'activités de loisirs\nCette sous-classe comprend la formation dans des activités sportives donnée à des groupes\nou des individus par des camps ou des écoles. Les écoles de formation sportive de jour et\ncelles offrant un hébergement sont également incluses. Cette sous-classe ne comprend pas\nla formation sportive donnée dans les écoles et universités. La formation peut être dispensée\ndans diverses structures telles que les installations de formation de l'unité ou du client, des\nétablissements d'enseignement ou par d'autres moyens. La formation relevant de cette sous-\nclasse est formellement organisée.\nCette sous-classe comprend :\n- la formation sportive (base-ball, basket-ball, cricket, football, etc.)\n- les camps offrant une formation sportive\n- les cours de gymnastique\n- les cours d'équitation donnés dans des académies ou écoles\n- les cours de natation\n- les instructeurs de sports, professeurs et entraîneurs professionnels

In [17]:
import polars as pl
import re
import pdfplumber

# ---------------------------------------------------------------------
# 3) NETTOYAGE DU BLOC (supprime numéros de page / section / etc.)
# ---------------------------------------------------------------------

def cleanup_naf_block(block):
    lines = block.split("\n")
    cleaned = []

    for line in lines:
        l = line.strip()

        if re.match(r"^\d+\s+Nomenclature", l):   break
        if re.match(r"^Section\s+[A-Z]\s*:", l):  break
        if re.match(r"^\d{2}\s+[A-Z]", l):        break
        if re.match(r"^\d{2}\.\d{1,2}$", l):      break

        cleaned.append(line)

    return "\n".join(cleaned).strip()

# ---------------------------------------------------------------------
# 4) PROGRAMME POLARS
# ---------------------------------------------------------------------

# Généré depuis le notebook 02
df = pl.read_csv("naf_codes_001.csv")

# Ajout colonne "desc" générée par extraction NAF
df = df.with_columns(
    pl.col("code_naf").map_elements(
        lambda code: cleanup_naf_block(
            extract_best_naf_block(full_text_pdf_naf, code)
        ).replace("\n", " "),  # mise sur une seule ligne
        return_dtype=pl.Utf8
    ).alias("desc")
)

# Sauvegarde du nouveau fichier
df.write_csv("naf_codes_001_desc.csv")
print("Fichier généré : naf_codes_001_desc.csv")
# Extrait du fichier CSV généré avec la description
df.sample(10)

Fichier généré : naf_codes_001_desc.csv


code_naf,code_rome,name,desc
str,str,str,str
"""64.20Z""","""""","""Activités des sociétés holding""","""64.20Z Activités des sociétés …"
"""81.30Z""","""""","""Services d'aménagement paysage…","""81.30.12 Autres services d'amé…"
"""41.10C""","""""","""Promotion immobilière d'autres…","""41.10C Promotion immobilière d…"
"""03.22Z""","""""","""Aquaculture en eau douce""","""03.22Z Aquaculture en eau douc…"
"""49.42Z""","""""","""Services de déménagement""","""49.42Z Services de déménagemen…"
"""47.91B""","""""","""Vente à distance sur catalogue…","""47.91B Vente à distance sur ca…"
"""23.64Z""","""""","""Fabrication de mortiers et bét…","""23.64.99 Opérations sous-trait…"
"""20.42Z""","""""","""Fabrication de parfums et de p…","""20.42Z Fabrication de parfums …"
"""14.12Z""","""""","""Fabrication de vêtements de tr…","""14.12.99 Opérations sous-trait…"
